<a href="https://colab.research.google.com/github/Tadashi1999/Repo_tst/blob/master/%E3%83%86%E3%82%AF%E3%83%8B%E3%82%AB%E3%83%AB%E3%83%81%E3%83%A3%E3%83%BC%E3%83%88%2B%E5%87%BA%E6%9D%A5%E9%AB%98v1_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Google Colabo & Talib & Plotlyで株価のテクニカルチャートを描く
!pip install pandas-datareader
#pandas-datareaderによる株価の取得方法１
import pandas as pd
import pandas_datareader as pdr
pdr=pdr.stooq.StooqDailyReader(symbols='4563.jp',start='Oct-01-2019',end="OCT-26-2020").read().sort_values(by='Date',ascending=True)
df_t=pdr


In [ ]:
df_t.tail(5)

In [ ]:
#Talibのインストール
! wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-lib
import talib as ta

In [ ]:
#ローソク足を簡易表示（実行可能）#何方かのパクリ、縦ラインの上で日付表示する
import numpy as np
import plotly.graph_objects as go

def plot_chart(df):
  x=np.arange(df.shape[0])
  interval=10
  vals=np.arange(df.shape[0],step=interval)
  labels=list(df[::interval].index)

  fig=go.Figure(
      data=[go.Candlestick(
            x=x,
            open=np.array(df['Open']),
            high=np.array(df['High']),
            low=np.array(df['Low']),
            close=np.array(df['Close']),
            hovertext=['Date']),
           ],
      layout=go.Layout(
            xaxis=dict(
                tickvals=vals,
                ticktext=labels,
                tickangle=-45
            ),
      )
  )
  fig.show()
plot_chart(df_t)  

In [ ]:
#複数のテクニカル分析
#akatak's blog,「PlotlとTalibでテクニカル分析チャートを描く」2019-11-23より、一部変更
import talib as ta
import numpy as np
import pandas as pd
!pip install plotly --upgrade

import plotly.offline as pyo
#import chart_studio.plotly as py
import plotly.graph_objects as go

#株価の取得方法２
import pandas_datareader.data as web
from datetime import datetime
start=datetime(2019,10,1)
end=datetime(2020,10,23)
df=web.DataReader('4563.jp','stooq',start,end)

#元原稿
#df=web.DataReader('^N225','yahoo','2019-10-01')
#df.tail()


hnp=np.array(df['High'])#npアレイ形式に変換
lnp=np.array(df['Low'])
cnp=np.array(df['Close'])
onp=np.array(df['Open'])
c=df['Close']#何故かMACDではこれで動作する


#単純移動平均
sma5=ta.SMA(c,timeperiod=2)
sma20=ta.SMA(c,timeperiod=10)
sma50=ta.SMA(c,timeperiod=50)

#Plotlyでは DataFrameを使うので、以下を追加
df['SMA5']=sma5
df['SMA20']=sma20
df['SMA50']=sma50

#MACD
macd,macd_sig,macd_hist=ta.MACD(c,fastperiod=5,slowperiod=10,signalperiod=9)
df["MACD"]=macd
df["MACD_SIG"]=macd_sig
df["MACD_HIST"]=macd_hist

#Plotlyでチャート
layout={
    'hight':1000,
    'title':{'text':'銘柄名','x':0.5},
    'titlefont':{'size':25},
    'xaxis':{'title':"",'rangeslider':{'visible':False}},
    'yaxis':{'domain':[.55,1],'title':"株価（円)",'side':"left",'tickformat':','},
    'yaxis2':{'domain':[.45,.55],'title':"MACD",'side':"right"},
}#akatak'sのまま

#ローソク足
trace1=go.Candlestick(
    x=df.index,open=onp,high=hnp,low=lnp,close=cnp,yaxis="y2",name="銘柄名"
)
trace2=go.Scatter(yaxis="y2",x=df.index,y=df["SMA5"],name='SMA5',line=dict(color='Black',width=1.5))
trace3=go.Scatter(yaxis="y2",x=df.index,y=df["SMA20"],name='SMA20',line=dict(color='Black',width=0.5))
trace4=go.Scatter(yaxis="y2",x=df.index,y=df["SMA50"],name='SMA50',line=dict(color='darkblue',width=1))
trace5=go.Scatter(yaxis="y2",x=df.index,y=df["MACD"],name='MACD',line=dict(color='blue',width=1.5))
trace6=go.Scatter(yaxis="y2",x=df.index,y=df["MACD_SIG"],name='MACD_SIG',line=dict(color='red',width=1))
trace7=go.Bar(yaxis="y2",x=df.index,y=np.array(df['Volume'])/50000,name="出来高")#高さ調整
data=[trace1,trace2,trace3,trace5,trace6,trace7]
go.Figure(data).show()#Google ColabではこれでOK

#課題　営業日のみの表示ができない

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.12.0)
